In [1]:
import sys,os,copy,glob
import numpy as np
sys.path.append(os.path.expanduser('~/smodels'))
from builder.protomodel import ProtoModel
from builder.manipulator import Manipulator
from tester.predictor import Predictor
from tester.combiner import Combiner
from walker.hiscore import Hiscore
from smodels.experiment.databaseObj import Database
from smodels.tools import runtime
from smodels.tools.physicsUnits import fb
runtime._experimental = True
import matplotlib.pyplot as plt
import seaborn as sns
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
sns.set_palette(sns.color_palette("Paired"))

In [2]:
def fromDict(inputDict):    
    p = ProtoModel(walkerid=0)
    for key,v in inputDict.items():
        setattr(p,key,copy.deepcopy(v))
    ma = Manipulator ( p )
    ma.initFromDict( inputDict )
    return p

In [3]:
#Get highest score from the run:
ff='./real3.dict'
with open( ff,'r') as f:
    pList = eval(f.read())
run = eval(os.path.basename(ff).replace('real','').replace('.dict',''))
pList = [fromDict(pDict) for pDict in pList[:]]
p = sorted(pList, key = lambda p: p.K, reverse=True)[0]
protomodel = p

[manipulator-12:26:56] starting with /home/walten/git/smodels-utils/protomodels/
[manipulator-12:26:56] starting with /home/walten/git/smodels-utils/protomodels/
[manipulator-12:26:56] starting with /home/walten/git/smodels-utils/protomodels/
[manipulator-12:26:56] starting with /home/walten/git/smodels-utils/protomodels/
[manipulator-12:26:56] starting with /home/walten/git/smodels-utils/protomodels/
[manipulator-12:26:56] starting with /home/walten/git/smodels-utils/protomodels/
[manipulator-12:26:56] starting with /home/walten/git/smodels-utils/protomodels/
[manipulator-12:26:56] starting with /home/walten/git/smodels-utils/protomodels/
[manipulator-12:26:56] starting with /home/walten/git/smodels-utils/protomodels/
[manipulator-12:26:56] starting with /home/walten/git/smodels-utils/protomodels/
[manipulator-12:26:56] starting with /home/walten/git/smodels-utils/protomodels/
[manipulator-12:26:56] starting with /home/walten/git/smodels-utils/protomodels/
[manipulator-12:26:56] start

In [29]:
protomodel = hi.hiscores[0]

In [30]:
# pred = Predictor(walkerid=0,dbpath='./smodels-database-noFastlim/')
pred = Predictor(walkerid=0,dbpath='./official123_smodels200.pcl')
res = pred.predict(protomodel)
oldPrior = pred.combiner.computePrior(protomodel,name='expo2')
newPrior = pred.combiner.computePrior(protomodel,name='expo1')

[combine:0] selected predictions down via SRs from 151 to 21.


[combine:0] sorting out subsets, reduced 3338 -> 405 combinations.


combination #405/405 100%|||||||||||||||||||||||||||||||||||||||||Time: 0:00:47


In [24]:
hi = Hiscore ( 0, False, "./hiscore.hi" )

[hiscore:0] loaded 1 hiscores from ./hiscore.hi.


In [26]:
Manipulator(hi.hiscores[0]).describe()


K = 8.77, Z = 3.51, muhat = 1.00, mumax = 1.0
	 Best Combo:
		 ATLAS-SUSY-2013-02 [T2] SR6jtp obsN=6 expBG=4.90+/-1.60 pred= 2.76E-05 [pb] UL = 3.99E-01 [fb]
		 ATLAS-SUSY-2013-15 [T2tt] tNboost obsN=5 expBG=3.30+/-0.70 pred= 4.53E-05 [pb] UL = 3.73E-01 [fb]
		 ATLAS-SUSY-2015-02 [T2tt] SR1 obsN=12 expBG=5.50+/-0.72 pred= 3.95E-04 [pb] UL = 4.40E+00 [fb]
		 ATLAS-SUSY-2016-07 [T2] 2j_Meff_1200 obsN=611 expBG=526.00+/-31.00 pred= 1.15E-03 [pb] UL = 4.84E+00 [fb]
		 CMS-SUS-13-012 [T2] None pred=6.39E-03 [pb], UL=3.80E+01 [fb]
		 CMS-SUS-16-050 [T2tt] None pred=4.60E-02 [pb], UL=1.79E+02 [fb]
	 Constraints:
		 r = 1.30 CMS-SUS-19-006 upperLimit [T2tt] theory xsec =  45.98, UL = 35.41, UL_exp = 55.04
		 r = 1.14 CMS-SUS-16-033 upperLimit [T2] theory xsec =  23.85, UL = 20.86, UL_exp = 30.81
		 r = 1.13 CMS-SUS-19-006 upperLimit [T2] theory xsec =  23.85, UL = 21.13, UL_exp = 23.69


In [31]:
ma.M.K

7.522839222989878